<a href="https://colab.research.google.com/github/sipocz/Blood/blob/main/blood_cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lazypredict


In [2]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

In [3]:
fname_url="https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip"
fname=fname_url.split("/")[-1]

In [4]:
!wget $fname_url


--2021-12-09 20:21:05--  https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip [following]
--2021-12-09 20:21:05--  https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16979362 (16M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  16.19M  --.-KB/s    in 0.1s    

2021-12-09 20:21:06 (131 MB/s) - ‘trai

In [5]:
!mkdir "train"
!rm ./train/Train.csv

mkdir: cannot create directory ‘train’: File exists


In [6]:
!unzip $fname -d "./train"
!rm train.zip

Archive:  train.zip
  inflating: ./train/Train.csv       


In [7]:
fname_train="./train/Train.csv"
df=pd.read_csv(fname_train)


In [8]:
def dew_point(T,RH):
    import math
    return 243.04*(math.log(RH/100)+((17.625*T)/(243.04+T)))/(17.625-math.log(RH/100)-((17.625*T)/(243.04+T)))
print(dew_point(45,10))

6.381492735225212


In [9]:
df["dewpoint"]=df.apply(lambda x:dew_point(x["temperature"],x["humidity"]),axis=1)

In [10]:
df.head(5)

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,dewpoint
0,ID_3SSHI56C,0.479669,0.477423,0.487956,0.491831,0.500516,0.502590,0.511561,0.514639,0.524245,0.536170,0.546407,0.561557,0.568417,0.571877,0.570884,0.569032,0.567476,0.565662,0.561901,0.559722,0.557474,0.554371,0.552386,0.548702,0.544238,0.542579,0.540514,0.538980,0.536650,0.536483,0.535447,0.537577,0.535715,0.536895,0.539589,0.541081,0.544893,0.547765,0.551773,...,1.462617,1.445696,1.435586,1.417847,1.404205,1.388861,1.377436,1.364444,1.360373,1.341243,1.339632,1.321471,1.317444,1.311209,1.291677,1.285579,1.285488,1.275784,1.271104,1.264029,1.250779,1.254856,1.255224,1.249623,1.244419,1.244437,1.243933,1.226790,1.234055,1.218660,1.213038,1.198317,1.195735,1.180846,42.51,34.01,ok,ok,ok,23.357237
1,ID_599OOLZA,0.471537,0.474113,0.479981,0.485528,0.491049,0.497942,0.504760,0.510543,0.522328,0.534423,0.548646,0.558420,0.565449,0.569717,0.570999,0.569969,0.568405,0.566628,0.564101,0.559951,0.556193,0.552271,0.550086,0.546207,0.542366,0.539789,0.537221,0.534336,0.533868,0.533018,0.532227,0.530818,0.532171,0.533658,0.535266,0.538939,0.542399,0.546479,0.550606,...,1.541997,1.533186,1.518359,1.498964,1.488043,1.472946,1.465925,1.452647,1.437819,1.423670,1.415103,1.401141,1.403560,1.384169,1.379410,1.374128,1.356969,1.352693,1.342430,1.339714,1.332805,1.336324,1.342537,1.332407,1.326258,1.336874,1.327538,1.311951,1.309399,1.304501,1.323005,1.305992,1.263887,1.262095,44.52,32.09,ok,high,high,24.133946
2,ID_MVJGPQ75,0.444998,0.458034,0.447386,0.456921,0.463225,0.475983,0.476817,0.481565,0.490010,0.505892,0.518125,0.530362,0.538530,0.543128,0.546287,0.547001,0.547120,0.546351,0.544254,0.542802,0.542207,0.539779,0.536417,0.533380,0.531117,0.529093,0.526101,0.524599,0.522952,0.521551,0.521149,0.520478,0.521432,0.521473,0.523567,0.525816,0.527889,0.530697,0.533416,...,1.502255,1.489132,1.483308,1.461028,1.453174,1.450412,1.437784,1.422148,1.415880,1.404698,1.388143,1.397241,1.385680,1.376355,1.357758,1.354854,1.345476,1.333810,1.332739,1.335550,1.326775,1.336862,1.316860,1.328051,1.328641,1.323526,1.314124,1.298936,1.289122,1.325059,1.271115,1.337119,1.289877,1.345229,45.77,24.80,ok,ok,high,20.952323
3,ID_CK6RF8YV,0.513434,0.513303,0.522609,0.521068,0.523146,0.530132,0.539517,0.546364,0.552414,0.565502,0.581143,0.594354,0.599457,0.604529,0.605267,0.606276,0.604895,0.603716,0.600683,0.598087,0.594303,0.589403,0.585883,0.581369,0.578962,0.575181,0.573274,0.570471,0.568241,0.565671,0.564579,0.563724,0.561978,0.562744,0.563455,0.565163,0.566505,0.569239,0.572075,...,1.423349,1.413718,1.403112,1.393964,1.375741,1.369549,1.354179,1.344562,1.333491,1.325002,1.321572,1.305561,1.292637,1.287971,1.283460,1.278300,1.268486,1.268407,1.263479,1.252612,1.254306,1.247635,1.242321,1.247859,1.246749,1.249920,1.265223,1.264013,1.285252,1.298422,1.299873,1.311157,1.303259,1.349833,45.84,36.93,low,ok,high,27.654980
4,ID_82N6QE6I,0.510485,0.519359,0.524225,0.528419,0.535273,0.545342,0.550314,0.557

In [11]:
df_dew_data=df.copy()
df_dew_data["dewpoint"]=df_dew_data.apply(lambda x:dew_point(x["temperature"],x["humidity"]),axis=1)

In [12]:
df_dew_data.head(80)


,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,dewpoint
0,ID_3SSHI56C,0.479669,0.477423,0.487956,0.491831,0.500516,0.502590,0.511561,0.514639,0.524245,0.536170,0.546407,0.561557,0.568417,0.571877,0.570884,0.569032,0.567476,0.565662,0.561901,0.559722,0.557474,0.554371,0.552386,0.548702,0.544238,0.542579,0.540514,0.538980,0.536650,0.536483,0.535447,0.537577,0.535715,0.536895,0.539589,0.541081,0.544893,0.547765,0.551773,...,1.462617,1.445696,1.435586,1.417847,1.404205,1.388861,1.377436,1.364444,1.360373,1.341243,1.339632,1.321471,1.317444,1.311209,1.291677,1.285579,1.285488,1.275784,1.271104,1.264029,1.250779,1.254856,1.255224,1.249623,1.244419,1.244437,1.243933,1.226790,1.234055,1.218660,1.213038,1.198317,1.195735,1.180846,42.51,34.01,ok,ok,ok,23.357237
1,ID_599OOLZA,0.471537,0.474113,0.479981,0.485528,0.491049,0.497942,0.504760,0.510543,0.522328,0.534423,0.548646,0.558420,0.565449,0.569717,0.570999,0.569969,0.568405,0.566628,0.564101,0.559951,0.556193,0.552271,0.550086,0.546207,0.542366,0.539789,0.537221,0.534336,0.533868,0.533018,0.532227,0.530818,0.532171,0.533658,0.535266,0.538939,0.542399,0.546479,0.550606,...,1.541997,1.533186,1.518359,1.498964,1.488043,1.472946,1.465925,1.452647,1.437819,1.423670,1.415103,1.401141,1.403560,1.384169,1.379410,1.374128,1.356969,1.352693,1.342430,1.339714,1.332805,1.336324,1.342537,1.332407,1.326258,1.336874,1.327538,1.311951,1.309399,1.304501,1.323005,1.305992,1.263887,1.262095,44.52,32.09,ok,high,high,24.133946
2,ID_MVJGPQ75,0.444998,0.458034,0.447386,0.456921,0.463225,0.475983,0.476817,0.481565,0.490010,0.505892,0.518125,0.530362,0.538530,0.543128,0.546287,0.547001,0.547120,0.546351,0.544254,0.542802,0.542207,0.539779,0.536417,0.533380,0.531117,0.529093,0.526101,0.524599,0.522952,0.521551,0.521149,0.520478,0.521432,0.521473,0.523567,0.525816,0.527889,0.530697,0.533416,...,1.502255,1.489132,1.483308,1.461028,1.453174,1.450412,1.437784,1.422148,1.415880,1.404698,1.388143,1.397241,1.385680,1.376355,1.357758,1.354854,1.345476,1.333810,1.332739,1.335550,1.326775,1.336862,1.316860,1.328051,1.328641,1.323526,1.314124,1.298936,1.289122,1.325059,1.271115,1.337119,1.289877,1.345229,45.77,24.80,ok,ok,high,20.952323
3,ID_CK6RF8YV,0.513434,0.513303,0.522609,0.521068,0.523146,0.530132,0.539517,0.546364,0.552414,0.565502,0.581143,0.594354,0.599457,0.604529,0.605267,0.606276,0.604895,0.603716,0.600683,0.598087,0.594303,0.589403,0.585883,0.581369,0.578962,0.575181,0.573274,0.570471,0.568241,0.565671,0.564579,0.563724,0.561978,0.562744,0.563455,0.565163,0.566505,0.569239,0.572075,...,1.423349,1.413718,1.403112,1.393964,1.375741,1.369549,1.354179,1.344562,1.333491,1.325002,1.321572,1.305561,1.292637,1.287971,1.283460,1.278300,1.268486,1.268407,1.263479,1.252612,1.254306,1.247635,1.242321,1.247859,1.246749,1.249920,1.265223,1.264013,1.285252,1.298422,1.299873,1.311157,1.303259,1.349833,45.84,36.93,low,ok,high,27.654980
4,ID_82N6QE6I,0.510485,0.519359,0.524225,0.528419,0.535273,0.545342,0.550314,0.557

In [13]:
def one_hot_encode_df(df,str_index):
    from sklearn.preprocessing import OneHotEncoder
    indf=df.copy(True)
    one_hot=OneHotEncoder()
    col=indf[str_index].values
    col=col.reshape(-1,1)
    one_hot.fit(col)
    out=one_hot.transform(col).toarray()
    codes=one_hot.get_feature_names()
    print(codes)
    for ind,code in enumerate(codes):
        print("Itt:",ind,codes[ind])
        indf[str_index+"_"+code]=out[:,ind]
    return(indf,one_hot)

In [14]:
df,hdl_onehot=one_hot_encode_df(df,"hdl_cholesterol_human")
df,hemo_onehot=one_hot_encode_df(df,"hemoglobin(hgb)_human")
df,col_onehot=one_hot_encode_df(df,"cholesterol_ldl_human")

['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok
['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok
['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok


In [15]:
df.tail()

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,dewpoint,hdl_cholesterol_human_x0_high,hdl_cholesterol_human_x0_low,hdl_cholesterol_human_x0_ok,hemoglobin(hgb)_human_x0_high,hemoglobin(hgb)_human_x0_low,hemoglobin(hgb)_human_x0_ok,cholesterol_ldl_human_x0_high,cholesterol_ldl_human_x0_low,cholesterol_ldl_human_x0_ok
13135,ID_NGPC0DA3,0.483513,0.482732,0.487531,0.497946,0.500697,0.505740,0.511904,0.519018,0.529455,0.540356,0.554324,0.563337,0.568932,0.573057,0.575291,0.574014,0.573149,0.570979,0.567632,0.564020,0.560878,0.557749,0.554752,0.552457,0.548313,0.547369,0.544267,0.542225,0.542338,0.539435,0.539836,0.539883,0.540450,0.541485,0.542836,0.544809,0.548269,0.552457,0.556129,...,1.292316,1.296431,1.281496,1.278615,1.268085,1.256004,1.255005,1.245078,1.245569,1.231796,1.228750,1.238677,1.230577,1.218419,1.224957,1.220593,1.218087,1.214382,1.208455,1.202338,1.189651,1.182641,1.206917,1.188173,1.188303,34.79,36.93,high,ok,ok,17.928970,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
13136,ID_XRBUD5U8,0.525435,0.527563,0.528863,0.531776,0.541156,0.547318,0.552549,0.559060,0.566892,0.581571,0.595248,0.603612,0.611313,0.614295,0.617476,0.617077,0.618908,0.616631,0.610841,0.604681,0.601588,0.598501,0.594149,0.589455,0.587673,0.584746,0.581093,0.580218,0.578071,0.577839,0.576454,0.576026,0.576220,0.579007,0.581564,0.584409,0.587786,0.591168,0.595874,...,1.457265,1.444710,1.452271,1.428153,1.421319,1.419508,1.411549,1.415561,1.405296,1.390187,1.384455,1.383000,1.380202,1.383244,1.378410,1.377784,1.359367,1.379885,1.378378,1.367057,1.388165,1.375223,1.372435,1.395317,1.387709,43.12,19.14,ok,ok,ok,14.656647,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
13137,ID_2M9L5NV2,0.512718,0.517815,0.524857,0.525466,0.536542,0.542930,0.550628,0.558939,0.567593,0.579192,0.595752,0.609749,0.617717,0.621270,0.625333,0.626705,0.623455,0.622948,0.618986,0.616963,0.614392,0.610574,0.606732,0.604206,0.601527,0.597425,0.595800,0.592601,0.590437,0.589152,0.588726,0.587134,0.588214,0.589699,0.590376,0.592544,0.594104,0.599264,0.603341,...,1.430362,1.427516,1.420605,1.414007,1.415796,1.395715,1.399176,1.392111,1.384322,1.376676,1.364141,1.363428,1.366804,1.371499,1.367140,1.364490,1.367668,1.362808,1.385546,1.386140,1.396862,1.377644,1.428150,1.447840,1.384221,42.48,43.41,ok,ok,ok,27.435633,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
13138,ID_C5V5SD2D,0.456747,0.472575,0.466935,0.466698,0.478546,0.486451,0.494838,0.496540,0.508274,0.521304,0.532652,0.542257,0.550854,0.556305,0.558084,0.559439,0.558418,0.554719,0.554852,0.552821,0.551641,0.549722,0.545233,0.542932,0.539894,0.536520,0.534783,0.530619,0.531076,0.530187,0.528713,0.527754,0.529230,0.529761,0.531586,0.533869,0.535897,0.539685,0.543911,...,1.287724,1.283147,1.269678,1.257537,1.259994,1.248000,1.253335,1.237789,1.228726,1.224605,1.224872,1.217346,1.225207,1.224785,1.214651,1.214882,1.214873,1.217364,1.208545,1.185192,1.181483,1.172430,1.174544,1.154095,1.212907,41.86,35.10,ok,ok,ok,23.312867,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
13139,ID_9R5ZR9NX,0.498372,0.501692,0.505924,0.510975,0.516707,0.520821,0.531242,0.537302,0.547303,0.563789,0.574825,0

In [16]:
df_list=pd.DataFrame()
df_list["data"]=df["hdl_cholesterol_human"]+"+"+  df["hemoglobin(hgb)_human"]+"+"+df["cholesterol_ldl_human"]

In [17]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

_3_Label_encoder=LabelEncoder()
_3_Label_encoder.fit(df_list)
_3_Label_encoder.classes_
y=_3_Label_encoder.transform(df_list)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [18]:
df_dew_data["y"]=y
del df_dew_data["Reading_ID"]

In [28]:
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(df_dew_data,x="humidity",y="dewpoint", color="y")
fig.show()

In [21]:
df_dew_data

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,dewpoint,y
0,0.479669,0.477423,0.487956,0.491831,0.500516,0.502590,0.511561,0.514639,0.524245,0.536170,0.546407,0.561557,0.568417,0.571877,0.570884,0.569032,0.567476,0.565662,0.561901,0.559722,0.557474,0.554371,0.552386,0.548702,0.544238,0.542579,0.540514,0.538980,0.536650,0.536483,0.535447,0.537577,0.535715,0.536895,0.539589,0.541081,0.544893,0.547765,0.551773,0.554599,...,1.445696,1.435586,1.417847,1.404205,1.388861,1.377436,1.364444,1.360373,1.341243,1.339632,1.321471,1.317444,1.311209,1.291677,1.285579,1.285488,1.275784,1.271104,1.264029,1.250779,1.254856,1.255224,1.249623,1.244419,1.244437,1.243933,1.226790,1.234055,1.218660,1.213038,1.198317,1.195735,1.180846,42.51,34.01,ok,ok,ok,23.357237,18
1,0.471537,0.474113,0.479981,0.485528,0.491049,0.497942,0.504760,0.510543,0.522328,0.534423,0.548646,0.558420,0.565449,0.569717,0.570999,0.569969,0.568405,0.566628,0.564101,0.559951,0.556193,0.552271,0.550086,0.546207,0.542366,0.539789,0.537221,0.534336,0.533868,0.533018,0.532227,0.530818,0.532171,0.533658,0.535266,0.538939,0.542399,0.546479,0.550606,0.554408,...,1.533186,1.518359,1.498964,1.488043,1.472946,1.465925,1.452647,1.437819,1.423670,1.415103,1.401141,1.403560,1.384169,1.379410,1.374128,1.356969,1.352693,1.342430,1.339714,1.332805,1.336324,1.342537,1.332407,1.326258,1.336874,1.327538,1.311951,1.309399,1.304501,1.323005,1.305992,1.263887,1.262095,44.52,32.09,ok,high,high,24.133946,12
2,0.444998,0.458034,0.447386,0.456921,0.463225,0.475983,0.476817,0.481565,0.490010,0.505892,0.518125,0.530362,0.538530,0.543128,0.546287,0.547001,0.547120,0.546351,0.544254,0.542802,0.542207,0.539779,0.536417,0.533380,0.531117,0.529093,0.526101,0.524599,0.522952,0.521551,0.521149,0.520478,0.521432,0.521473,0.523567,0.525816,0.527889,0.530697,0.533416,0.540124,...,1.489132,1.483308,1.461028,1.453174,1.450412,1.437784,1.422148,1.415880,1.404698,1.388143,1.397241,1.385680,1.376355,1.357758,1.354854,1.345476,1.333810,1.332739,1.335550,1.326775,1.336862,1.316860,1.328051,1.328641,1.323526,1.314124,1.298936,1.289122,1.325059,1.271115,1.337119,1.289877,1.345229,45.77,24.80,ok,ok,high,20.952323,16
3,0.513434,0.513303,0.522609,0.521068,0.523146,0.530132,0.539517,0.546364,0.552414,0.565502,0.581143,0.594354,0.599457,0.604529,0.605267,0.606276,0.604895,0.603716,0.600683,0.598087,0.594303,0.589403,0.585883,0.581369,0.578962,0.575181,0.573274,0.570471,0.568241,0.565671,0.564579,0.563724,0.561978,0.562744,0.563455,0.565163,0.566505,0.569239,0.572075,0.575452,...,1.413718,1.403112,1.393964,1.375741,1.369549,1.354179,1.344562,1.333491,1.325002,1.321572,1.305561,1.292637,1.287971,1.283460,1.278300,1.268486,1.268407,1.263479,1.252612,1.254306,1.247635,1.242321,1.247859,1.246749,1.249920,1.265223,1.264013,1.285252,1.298422,1.299873,1.311157,1.303259,1.349833,45.84,36.93,low,ok,high,27.654980,9
4,0.510485,0.519359,0.524225,0.528419,0.535273,0.545342,0.550314,0.557129,0.567030,0.577731,0.589192,0.604401,0.611372,0.614571,0

In [22]:
del df_dew_data["hdl_cholesterol_human"]
del df_dew_data["hemoglobin(hgb)_human"]
del df_dew_data["cholesterol_ldl_human"]




In [114]:
from sklearn.cluster import DBSCAN, KMeans
dbscan = DBSCAN(eps=0.4, min_samples=5).fit(df_dew_data)
df_dew_data["dbscan"]=dbscan.labels_

In [107]:
max(dbscan.labels_)

792

In [74]:

df_dew_data.head()

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,dewpoint,y,dbscan
0,0.479669,0.477423,0.487956,0.491831,0.500516,0.502590,0.511561,0.514639,0.524245,0.536170,0.546407,0.561557,0.568417,0.571877,0.570884,0.569032,0.567476,0.565662,0.561901,0.559722,0.557474,0.554371,0.552386,0.548702,0.544238,0.542579,0.540514,0.538980,0.536650,0.536483,0.535447,0.537577,0.535715,0.536895,0.539589,0.541081,0.544893,0.547765,0.551773,0.554599,...,1.469838,1.462617,1.445696,1.435586,1.417847,1.404205,1.388861,1.377436,1.364444,1.360373,1.341243,1.339632,1.321471,1.317444,1.311209,1.291677,1.285579,1.285488,1.275784,1.271104,1.264029,1.250779,1.254856,1.255224,1.249623,1.244419,1.244437,1.243933,1.226790,1.234055,1.218660,1.213038,1.198317,1.195735,1.180846,42.51,34.01,23.357237,18,0
1,0.471537,0.474113,0.479981,0.485528,0.491049,0.497942,0.504760,0.510543,0.522328,0.534423,0.548646,0.558420,0.565449,0.569717,0.570999,0.569969,0.568405,0.566628,0.564101,0.559951,0.556193,0.552271,0.550086,0.546207,0.542366,0.539789,0.537221,0.534336,0.533868,0.533018,0.532227,0.530818,0.532171,0.533658,0.535266,0.538939,0.542399,0.546479,0.550606,0.554408,...,1.552979,1.541997,1.533186,1.518359,1.498964,1.488043,1.472946,1.465925,1.452647,1.437819,1.423670,1.415103,1.401141,1.403560,1.384169,1.379410,1.374128,1.356969,1.352693,1.342430,1.339714,1.332805,1.336324,1.342537,1.332407,1.326258,1.336874,1.327538,1.311951,1.309399,1.304501,1.323005,1.305992,1.263887,1.262095,44.52,32.09,24.133946,12,1
2,0.444998,0.458034,0.447386,0.456921,0.463225,0.475983,0.476817,0.481565,0.490010,0.505892,0.518125,0.530362,0.538530,0.543128,0.546287,0.547001,0.547120,0.546351,0.544254,0.542802,0.542207,0.539779,0.536417,0.533380,0.531117,0.529093,0.526101,0.524599,0.522952,0.521551,0.521149,0.520478,0.521432,0.521473,0.523567,0.525816,0.527889,0.530697,0.533416,0.540124,...,1.516723,1.502255,1.489132,1.483308,1.461028,1.453174,1.450412,1.437784,1.422148,1.415880,1.404698,1.388143,1.397241,1.385680,1.376355,1.357758,1.354854,1.345476,1.333810,1.332739,1.335550,1.326775,1.336862,1.316860,1.328051,1.328641,1.323526,1.314124,1.298936,1.289122,1.325059,1.271115,1.337119,1.289877,1.345229,45.77,24.80,20.952323,16,0
3,0.513434,0.513303,0.522609,0.521068,0.523146,0.530132,0.539517,0.546364,0.552414,0.565502,0.581143,0.594354,0.599457,0.604529,0.605267,0.606276,0.604895,0.603716,0.600683,0.598087,0.594303,0.589403,0.585883,0.581369,0.578962,0.575181,0.573274,0.570471,0.568241,0.565671,0.564579,0.563724,0.561978,0.562744,0.563455,0.565163,0.566505,0.569239,0.572075,0.575452,...,1.442957,1.423349,1.413718,1.403112,1.393964,1.375741,1.369549,1.354179,1.344562,1.333491,1.325002,1.321572,1.305561,1.292637,1.287971,1.283460,1.278300,1.268486,1.268407,1.263479,1.252612,1.254306,1.247635,1.242321,1.247859,1.246749,1.249920,1.265223,1.264013,1.285252,1.298422,1.299873,1.311157,1.303259,1.349833,45.84,36.93,27.654980,9,1
4,0.510485,0.519359,0.524225,0.528419,0.535273,0.545342,0.550314,0.557129,0.567030,0.577731,0.589192,0.604401,0.611372,0.6145

In [108]:
import plotly.express as px
fig = px.scatter(df_dew_data,x="humidity",y="dewpoint", color="dbscan",labels="y")
fig.show()

In [131]:
kmeans = KMeans(n_clusters=300).fit(df_dew_data)


In [132]:
max(kmeans.labels_)

299

In [133]:
df_dew_data["kmeans"]=kmeans.labels_

In [134]:
import plotly.express as px
fig = px.scatter(df_dew_data,x="humidity",y="dewpoint", color="kmeans")
fig.show()

In [145]:
func=["mean"]
df


In [168]:
df_agg=pd.DataFrame()
for col in df_dew_data.columns:
    print(col)
    result=df_dew_data.groupby("kmeans").agg({col:func})
    df_agg.append(result,ignore_index=True)

absorbance0
absorbance1
absorbance2
absorbance3
absorbance4
absorbance5
absorbance6
absorbance7
absorbance8
absorbance9
absorbance10
absorbance11
absorbance12
absorbance13
absorbance14
absorbance15
absorbance16
absorbance17
absorbance18
absorbance19
absorbance20
absorbance21
absorbance22
absorbance23
absorbance24
absorbance25
absorbance26
absorbance27
absorbance28
absorbance29
absorbance30
absorbance31
absorbance32
absorbance33
absorbance34
absorbance35
absorbance36
absorbance37
absorbance38
absorbance39
absorbance40
absorbance41
absorbance42
absorbance43
absorbance44
absorbance45
absorbance46
absorbance47
absorbance48
absorbance49
absorbance50
absorbance51
absorbance52
absorbance53
absorbance54
absorbance55
absorbance56
absorbance57
absorbance58
absorbance59
absorbance60
absorbance61
absorbance62
absorbance63
absorbance64
absorbance65
absorbance66
absorbance67
absorbance68
absorbance69
absorbance70
absorbance71
absorbance72
absorbance73
absorbance74
absorbance75
absorbance76
absorbanc

In [169]:
df_agg

""


In [33]:

del df["hdl_cholesterol_human"]
del df["hemoglobin(hgb)_human"]
del df["cholesterol_ldl_human"]




In [34]:
from lazypredict.Supervised import LazyRegressor
from lazypredict.Supervised import LazyClassifier

from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:143: FutureWarning:

The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.



In [35]:
col_names=df.columns[-9:]

In [36]:
col_names[0]

'hdl_cholesterol_human_x0_high'

In [37]:
X=df.iloc[:,1:-9]
y

array([18, 12, 16, ..., 18, 18,  2])

In [38]:
y

array([18, 12, 16, ..., 18, 18,  2])

In [39]:
X

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,dewpoint
0,0.48,0.48,0.49,0.49,0.50,0.50,0.51,0.51,0.52,0.54,0.55,0.56,0.57,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.55,...,1.51,1.49,1.47,1.46,1.45,1.44,1.42,1.40,1.39,1.38,1.36,1.36,1.34,1.34,1.32,1.32,1.31,1.29,1.29,1.29,1.28,1.27,1.26,1.25,1.25,1.26,1.25,1.24,1.24,1.24,1.23,1.23,1.22,1.21,1.20,1.20,1.18,42.51,34.01,23.36
1,0.47,0.47,0.48,0.49,0.49,0.50,0.50,0.51,0.52,0.53,0.55,0.56,0.57,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.54,0.54,0.54,0.55,0.55,0.55,...,1.59,1.58,1.55,1.54,1.53,1.52,1.50,1.49,1.47,1.47,1.45,1.44,1.42,1.42,1.40,1.40,1.38,1.38,1.37,1.36,1.35,1.34,1.34,1.33,1.34,1.34,1.33,1.33,1.34,1.33,1.31,1.31,1.30,1.32,1.31,1.26,1.26,44.52,32.09,24.13
2,0.44,0.46,0.45,0.46,0.46,0.48,0.48,0.48,0.49,0.51,0.52,0.53,0.54,0.54,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.53,0.53,0.53,0.53,0.54,...,1.54,1.53,1.52,1.50,1.49,1.48,1.46,1.45,1.45,1.44,1.42,1.42,1.40,1.39,1.40,1.39,1.38,1.36,1.35,1.35,1.33,1.33,1.34,1.33,1.34,1.32,1.33,1.33,1.32,1.31,1.30,1.29,1.33,1.27,1.34,1.29,1.35,45.77,24.80,20.95
3,0.51,0.51,0.52,0.52,0.52,0.53,0.54,0.55,0.55,0.57,0.58,0.59,0.60,0.60,0.61,0.61,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.56,0.56,0.57,0.57,0.57,0.57,0.58,...,1.47,1.46,1.44,1.42,1.41,1.40,1.39,1.38,1.37,1.35,1.34,1.33,1.33,1.32,1.31,1.29,1.29,1.28,1.28,1.27,1.27,1.26,1.25,1.25,1.25,1.24,1.25,1.25,1.25,1.27,1.26,1.29,1.30,1.30,1.31,1.30,1.35,45.84,36.93,27.65
4,0.51,0.52,0.52,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.59,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.60,0.60,0.60,0.60,0.60,0.61,0.61,0.61,...,1.46,1.45,1.43,1.43,1.42,1.41,1.41,1.40,1.39,1.38,1.37,1.36,1.35,1.35,1.35,1.33,1.33,1.32,1.32,1.32,1.30,1.30,1.29,1.30,1.29,1.29,1.29,1.29,1.29,1.30,1.27,1.27,1.27,1.29,1.34,1.32,1.35,38.92,23.88,14.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13135,0.48,0.48,0.49,0.50,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.57,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.56,0.56,...,1.43,1.41,1.41,1.39,1.38,1.37,1.36,1.34,1.33,1.33,1.31,1.30,1.29,1.30,1.28,1.28,1.27,1.26,1.26,1.25,1.25,1.23,1.23,1.24,1.23,1.22,1.22,1.22,1.22,1.21,1.21,1.20,1.19,1.18,1.21,1.19,1.19,34.79,36.93,17.93
13136,0.53,0.53,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.60,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.58,

In [40]:
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(df_ordered,x="humidity",y="dewpoint", color=)
fig.show()

In [ ]:
offset=int(0.9*len(X))
print(offset)

11826


In [ ]:
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,predictions=True)
modells,predictions = clf.fit(X_train, X_test, y_train, y_test)


  3%|▎         | 1/29 [00:19<08:54, 19.08s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.3082191780821918, 'Balanced Accuracy': 0.0881161713107252, 'ROC AUC': None, 'F1 Score': 0.20457526872083198, 'Time taken': 19.08021378517151}


  7%|▋         | 2/29 [00:41<09:33, 21.25s/it]

{'Model': 'BaggingClassifier', 'Accuracy': 0.9885844748858448, 'Balanced Accuracy': 0.9675056402573597, 'ROC AUC': None, 'F1 Score': 0.9885110699693824, 'Time taken': 22.758439540863037}


 10%|█         | 3/29 [00:42<05:02, 11.65s/it]

{'Model': 'BernoulliNB', 'Accuracy': 0.1050228310502283, 'Balanced Accuracy': 0.4000476219206964, 'ROC AUC': None, 'F1 Score': 0.07975450452560849, 'Time taken': 0.22481536865234375}


 10%|█         | 3/29 [02:32<22:02, 50.85s/it]


KeyboardInterrupt: ignored

In [ ]:
modells

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LabelSpreading,1.00,1.00,1.00,1.00,10.72
KNeighborsClassifier,1.00,1.00,1.00,1.00,0.96
LabelPropagation,1.00,1.00,1.00,1.00,6.49
ExtraTreesClassifier,1.00,1.00,1.00,1.00,2.25
LGBMClassifier,1.00,0.99,0.99,1.00,4.69
XGBClassifier,0.99,0.98,0.98,0.99,19.90
RandomForestClassifier,0.99,0.98,0.98,0.99,14.24
BaggingClassifier,0.99,0.98,0.98,0.99,18.96
ExtraTreeClassifier,0.98,0.97,0.97,0.98,0.18


In [ ]:
from sklearn.neighbors import  KNeighborsClassifier
t1=KNeighborsClassifier()
o=t1.fit(X_train,y_train)

In [ ]:
y_train

array([18, 12, 16, ..., 18, 16, 18])

In [ ]:
o.predict

<bound method KNeighborsClassifier.predict of KNeighborsClassifier()>

In [ ]:

my_cl=[]

for i in range(9):

    my_cl.append(  KNeighborsClassifier())
    X=df.iloc[:,1:-9]
    y=df.loc[:,col_names[i]]
    my_cl[i].fit(X,y)



In [ ]:
o.predict(X_train)

array([18, 12, 16, ..., 18, 16, 18])

In [ ]:
X[0:0]

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity


In [ ]:
my_cl

[KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier()]

In [ ]:
my_cl[0].predict(X)

array([0., 0., 0., ..., 0., 0., 1.])

In [ ]:
my_cl[1].predict(X)

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
my_cl[2].predict(X)

array([1., 1., 1., ..., 1., 1., 0.])

In [ ]:
# submission generation
!rm "./Test.csv"

In [ ]:
!wget https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv

--2021-12-07 22:18:47--  https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv [following]
--2021-12-07 22:18:48--  https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10299206 (9.8M) [text/plain]
Saving to: ‘Test.csv’

Test.csv            100%[===================>]   9.82M  --.-KB/s    in 0.09s   

2021-12-07 22:18:48 (114 MB/s) - ‘Test.csv’ sa

In [ ]:
test_df=pd.read_csv("./Test.csv")
test_df.head()

test_df['hdl_cholesterol_human']=0
test_df["hemoglobin(hgb)_human"]=0
test_df["cholesterol_ldl_human"]=0

In [ ]:
X_test=test_df.iloc[:,1:-3]

In [ ]:
X_test[0:0]

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity


In [ ]:
from sklearn.neighbors import  KNeighborsClassifier


for i in range(9):

    y_pred=my_cl[i].predict(X_test)
    test_df[col_names[i]]=y_pred
    



In [ ]:
y_pred2=o.predict(X_test)

In [ ]:
test_df

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,hdl_cholesterol_human_x0_high,hdl_cholesterol_human_x0_low,hdl_cholesterol_human_x0_ok,hemoglobin(hgb)_human_x0_high,hemoglobin(hgb)_human_x0_low,hemoglobin(hgb)_human_x0_ok,cholesterol_ldl_human_x0_high,cholesterol_ldl_human_x0_low,cholesterol_ldl_human_x0_ok
0,ID_37BEI22R,0.45,0.45,0.45,0.46,0.47,0.49,0.49,0.50,0.50,0.51,0.53,0.54,0.55,0.56,0.56,0.56,0.56,0.56,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.55,...,1.32,1.32,1.31,1.31,1.31,1.30,1.29,1.29,1.29,1.29,1.29,1.27,1.27,1.27,1.26,1.25,1.25,1.25,1.22,1.20,1.21,1.16,1.15,1.17,1.16,1.20,40.88,25.70,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00
1,ID_4W85V5DV,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.56,0.58,0.59,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0.58,0.58,0.59,0.59,0.59,0.60,0.60,...,1.43,1.41,1.40,1.40,1.39,1.38,1.37,1.37,1.36,1.36,1.35,1.35,1.33,1.33,1.34,1.32,1.32,1.32,1.31,1.29,1.27,1.24,1.24,1.22,1.25,1.26,42.35,30.16,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
2,ID_L4YR3NDY,0.44,0.44,0.44,0.45,0.46,0.47,0.47,0.48,0.48,0.50,0.51,0.52,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.52,0.52,0.52,0.51,0.51,0.51,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.51,...,1.25,1.24,1.23,1.22,1.21,1.21,1.20,1.20,1.20,1.19,1.18,1.17,1.17,1.17,1.17,1.16,1.16,1.16,1.16,1.16,1.16,1.16,1.16,1.16,1.18,1.17,42.83,25.51,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00
3,ID_U88E3SQ6,0.50,0.51,0.51,0.52,0.53,0.54,0.54,0.54,0.55,0.56,0.57,0.58,0.59,0.59,0.60,0.60,0.60,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.56,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.55,0.56,0.56,0.56,0.56,0.57,0.57,...,1.29,1.28,1.27,1.26,1.26,1.25,1.24,1.23,1.23,1.21,1.22,1.21,1.21,1.20,1.20,1.20,1.20,1.18,1.17,1.14,1.11,1.05,1.05,1.02,1.03,1.02,40.95,41.32,0,0,0,0.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00
4,ID_NW7Z3XU7,0.53,0.53,0.54,0.54,0.55,0.56,0.56,0.57,0.58,0.59,0.60,0.62,0.62,0.63,0.63,0.63,0.63,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.60,0.60,0.60,...,1.40,1.39,1.39,1.38,1.36,1.35,1.35,1.34,1.33,1.33,1.33,1.31,1.32,1.30,1.30,1.31,1.31,1.31,1.29,1.29,1.30,1.29,1.27,1.28,1.25,1.29,46.99,24.21,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655,ID_ADCBL266,0.51,0.51,0.51,0.51,0.52,0.52,0.53,0.54,0.55,0.56,0.57,0.59,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.57,0.57,0.58,0.58,0.58,0.59,0.59,...,1.47,1.45,1.45,1.43,1.41,1.42,1.40,1.39,1.39,1.38,1.37,1.36,1.36,1.36,1.34,1.35,1.37,1.35,1.37,1.35,1.37,1.36,1.38,1.37,1.35,1.31,35.54,37.52,0,0,0,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
3656,ID_

In [ ]:
hdl=test_df.loc[:,"hdl_cholesterol_human_x0_high":	"hdl_cholesterol_human_x0_ok"]
hemo=test_df.loc[:,"hemoglobin(hgb)_human_x0_high":	"hemoglobin(hgb)_human_x0_ok"]
col=test_df.loc[:,"cholesterol_ldl_human_x0_high":"cholesterol_ldl_human_x0_ok"]


In [ ]:
hdl_deconv=hdl_onehot.inverse_transform(hdl)
hemo_deconv=hemo_onehot.inverse_transform(hemo)
col_deconv=col_onehot.inverse_transform(col)


In [ ]:
y_pred_deconv=_3_Label_encoder.inverse_transform(y_pred2)

In [ ]:
y_pred_deconv_list=[i.split("+") for i in y_pred_deconv]

In [ ]:
y_pred_deconv_list

[['ok', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['ok', 'ok', 'high'],
 ['low', 'ok', 'high'],
 ['ok', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['ok', 'ok', 'ok'],
 ['ok', 'ok', 'high'],
 ['high', 'ok', 'high'],
 ['ok', 'high', 'high'],
 ['low', 'ok', 'high'],
 ['ok', 'ok', 'high'],
 ['ok', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'high', 'ok'],
 ['ok', 'ok', 'ok'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'ok', 'high'],
 ['high', 'ok', 'high'],
 ['high', 'ok', 'high'],
 ['high', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'ok'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'low', 'ok'],
 ['ok', 'high', 'ok'],
 ['ok', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'ok'],
 ['low', 'ok', 'ok'],
 ['low', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'high', 'ok'],
 ['ok', 'ok', 'high'],
 ['low', 'ok', 'ok'],
 ['low', 'ok', 'ok'],
 

In [ ]:
y_pred_deconv_list[:][0]

['ok', 'ok', 'high']

In [ ]:
o_df_2=pd.DataFrame(y_pred_deconv_list)

In [ ]:
o_df_2.

,0,1,2
0,ok,ok,high
1,ok,ok,ok
2,ok,ok,high
3,low,ok,high
4,ok,ok,high


In [ ]:
test_df["hdl_cholesterol_human"]=o_df_2[0]
test_df["hemoglobin(hgb)_human"]=o_df_2[1]
test_df["cholesterol_ldl_human"]=o_df_2[2]



In [ ]:
!rm SampleSubmission.csv
!wget https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv

--2021-12-07 22:33:24--  https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv [following]
--2021-12-07 22:33:24--  https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 384318 (375K) [text/plain]
Saving to: ‘SampleSubmission.csv’

SampleSubmission.cs 100%[===================>] 375.31K  --.-KB/s    in 0.02s   



In [ ]:
submission_df=pd.read_csv("./SampleSubmission.csv")

In [ ]:
submission_df.head(1)

,Reading_ID,target
0,ID_00902R9H_hdl_cholesterol_human,nan


In [ ]:
outdf=test_df
outdf.set_index("Reading_ID",inplace=True)

In [ ]:
outdf.loc["ID_UPO13B7Z"]

absorbance0                     0.49
absorbance1                     0.50
absorbance2                     0.50
absorbance3                     0.51
absorbance4                     0.52
                                ... 
hemoglobin(hgb)_human_x0_low    0.00
hemoglobin(hgb)_human_x0_ok     1.00
cholesterol_ldl_human_x0_high   0.00
cholesterol_ldl_human_x0_low    0.00
cholesterol_ldl_human_x0_ok     1.00
Name: ID_UPO13B7Z, Length: 184, dtype: object

In [ ]:
submission_df.head(1)

,Reading_ID,target
0,ID_00902R9H_hdl_cholesterol_human,nan


In [ ]:
for i in submission_df.index:
    #print(i)
    indexer=submission_df.loc[i,"Reading_ID"]

    splitter=indexer.split("_",2)
    out_index=splitter[0]+"_"+splitter[1]
    col_index=splitter[2]
    print(out_index,col_index)
    result=outdf.loc[out_index][col_index]
    submission_df.iloc[i,1]=result

A streamkimeneten csak az utolsó 5000 sor látható.
ID_JVY48D8U cholesterol_ldl_human
ID_JVY48D8U hemoglobin(hgb)_human
ID_JW4F66XM cholesterol_ldl_human
ID_JW4F66XM hdl_cholesterol_human
ID_JW4F66XM hemoglobin(hgb)_human
ID_JWXML3KU cholesterol_ldl_human
ID_JWXML3KU hemoglobin(hgb)_human
ID_JWXML3KU hdl_cholesterol_human
ID_JXCWA9OK cholesterol_ldl_human
ID_JXCWA9OK hemoglobin(hgb)_human
ID_JXCWA9OK hdl_cholesterol_human
ID_JY2WRCO4 hemoglobin(hgb)_human
ID_JY2WRCO4 hdl_cholesterol_human
ID_JY2WRCO4 cholesterol_ldl_human
ID_JYFHLCK2 cholesterol_ldl_human
ID_JYFHLCK2 hemoglobin(hgb)_human
ID_JYFHLCK2 hdl_cholesterol_human
ID_JYUWNU78 cholesterol_ldl_human
ID_JYUWNU78 hemoglobin(hgb)_human
ID_JYUWNU78 hdl_cholesterol_human
ID_JYV38755 hemoglobin(hgb)_human
ID_JYV38755 cholesterol_ldl_human
ID_JYV38755 hdl_cholesterol_human
ID_JYYFUL4N cholesterol_ldl_human
ID_JYYFUL4N hdl_cholesterol_human
ID_JYYFUL4N hemoglobin(hgb)_human
ID_JZ2BEIAY cholesterol_ldl_human
ID_JZ2BEIAY hdl_cholesterol_hum

In [ ]:
submission_df.set_index("Reading_ID", inplace=True)

In [ ]:
submission_df

,target
Reading_ID,
ID_00902R9H_hdl_cholesterol_human,low
ID_00902R9H_hemoglobin(hgb)_human,ok
ID_00902R9H_cholesterol_ldl_human,ok
ID_00FQS7AO_hdl_cholesterol_human,ok
ID_00FQS7AO_hemoglobin(hgb)_human,ok
...,...
ID_ZZKK303Z_hdl_cholesterol_human,ok
ID_ZZKK303Z_cholesterol_ldl_human,ok
ID_ZZRYJBQJ_hdl_cholesterol_human,high


In [ ]:

    

def create_submission_name(project):
    from datetime  import datetime
    strdate=datetime.now().strftime("%Y%m%d_%H%M%S")
    out=project+"_"+"submission"+"_"+strdate+".csv"
    return(out)




In [ ]:
submission_df.head()

,target
Reading_ID,
ID_00902R9H_hdl_cholesterol_human,low
ID_00902R9H_hemoglobin(hgb)_human,ok
ID_00902R9H_cholesterol_ldl_human,ok
ID_00FQS7AO_hdl_cholesterol_human,ok
ID_00FQS7AO_hemoglobin(hgb)_human,ok


In [ ]:
submission_fname=create_submission_name("BloodAI")
print(submission_fname)


BloodAI_submission_20211207_223425.csv


In [ ]:
submission_df.to_csv(submission_fname,index=True)


In [ ]:
!head $submission_fname

Reading_ID,target
ID_00902R9H_hdl_cholesterol_human,low
ID_00902R9H_hemoglobin(hgb)_human,ok
ID_00902R9H_cholesterol_ldl_human,ok
ID_00FQS7AO_hdl_cholesterol_human,ok
ID_00FQS7AO_hemoglobin(hgb)_human,ok
ID_00FQS7AO_cholesterol_ldl_human,ok
ID_00QJTIBO_hdl_cholesterol_human,low
ID_00QJTIBO_hemoglobin(hgb)_human,ok
ID_00QJTIBO_cholesterol_ldl_human,ok
